# Data Cleaning

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out". Feeding dirty data into a model will give us results that are meaningless.

Specifically, we'll be walking through:

1. **Getting the data - **in this case, we'll be scraping data from a website
2. **Cleaning the data - **we will walk through popular text pre-processing techniques
3. **Organizing the data - **we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format

## Problem Statement

As a reminder, our goal is to look at transcripts of various comedians and note their similarities and differences. Specifically, I'd like to know if Ali Wong's comedy style is different than other comedians, since she's the comedian that got me interested in stand up comedy.

## Getting The Data

Luckily, there are wonderful people online that keep track of stand up routine transcripts. [Scraps From The Loft](http://scrapsfromtheloft.com) makes them available for non-profit and educational purposes.

To decide which comedians to look into, I went on IMDB and looked specifically at comedy specials that were released in the past 5 years. To narrow it down further, I looked only at those with greater than a 7.5/10 rating and more than 2000 votes. If a comedian had multiple specials that fit those requirements, I would pick the most highly rated one. I ended up with a dozen comedy specials.

In [40]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    for page in range(1, 18):
        page = requests.get(url, params={'pgno': page}).text
        soup = BeautifulSoup(page, "lxml")
        links = soup.select("h2.f18 a")
        for a in links:
            print(a["href"])
    
# URLs of transcripts in scope
url = ['https://www.thestar.com.my/search?q=Development%20Financial%20Institutions']


In [41]:
# # Actually request transcripts (takes a few minutes to run)
transcripts = [url_to_transcript(u) for u in url]

https://www.thestar.com.my/news/world/2020/05/29/as-lockdowns-ease-a-bronx-nail-salon-plans-a-careful-reopening
https://www.thestar.com.my/business/business-news/2020/04/24/us-fed-says-working-to-broaden-access-to-039ppp039-facility-beyond-banks
https://www.thestar.com.my/business/business-news/2020/04/18/bank-negara-to-ensure-sufficient-ringgit-liquidity
https://www.thestar.com.my/business/business-news/2020/04/18/providing-sme-support-in-times-of-need
https://www.thestar.com.my/business/business-news/2020/04/17/bank-negara-ensures-sufficient-ringgit-liquidity-in-onshore-market
https://www.thestar.com.my/news/nation/2020/03/30/why-are-not-all-glcs-roped-in-to-contribute
https://www.thestar.com.my/business/business-news/2020/03/26/moratorium-is-automatic-says-bank-negara
https://www.thestar.com.my/business/business-news/2020/03/25/bnm-no-need-to-apply-interest-will-be-accrued
https://www.thestar.com.my/news/nation/2020/03/21/bank-negara-releases-faq
https://www.thestar.com.my/business/

https://www.thestar.com.my/news/nation/2015/01/21/more-funds-for-floodhit-areas-rm893mil-to-be-set-aside-under-budget-2015-for-flood-mitigation-projec
https://www.thestar.com.my/news/nation/2015/01/20/budget-rivision-full-speech
https://www.thestar.com.my/business/business-news/2014/10/23/husni-urges-multilateral-organisations-to-consolidate-their-expertise-resources
https://www.thestar.com.my/business/business-news/2014/10/16/bank-negara-seeks-feedback
https://www.thestar.com.my/business/business-news/2014/10/11/household-loan-approvals-continue-to-grow
https://www.thestar.com.my/business/business-news/2014/10/11/us-policy-will-still-hold-sway-monetary-measures-expected-to-impact-malaysia-and-emerging-economies
https://www.thestar.com.my/business/business-news/2014/05/13/financial-services-5-critical-targets-launch-of-a-crowdfunding-platform-tops-list
https://www.thestar.com.my/business/business-news/2013/11/06/comfortable-household-debt-level-it-wont-affect-banks-creditworthiness-mea

https://www.thestar.com.my/business/business-news/2003/12/15/banking-on-nation-building
https://www.thestar.com.my/business/business-news/2003/10/31/27-growth-target-for-agriculture
https://www.thestar.com.my/business/business-news/2003/09/24/greater-efforts-needed--to-boost-islamic-financing
https://www.thestar.com.my/business/business-news/2003/09/13/challenges-in-fiscal-consolidation-process
https://www.thestar.com.my/business/business-news/2003/08/12/bank-industri-posts-record-pretax-profit
https://www.thestar.com.my/business/business-news/2003/06/12/public-bank-govt-stimulus-impact-positive
https://www.thestar.com.my/business/business-news/2003/05/28/new-3phase-5year-plan-to-transform-bsn
https://www.thestar.com.my/news/nation/2003/05/22/government-unveils-economic-stimulus-package
https://www.thestar.com.my/business/business-news/2003/05/21/benchmark-ops-fmm-tells-banks
https://www.thestar.com.my/business/business-news/2003/05/20/bank-negara-sets-up-sme-special-unit


In [18]:
# # Pickle files for later use

# # Make a new directory to hold the text files
# !mkdir transcripts

# for i, c in enumerate(comedians):
#     with open("transcripts/" + c + ".txt", "wb") as file:
#         pickle.dump(transcripts[i], file)

In [ ]:
# Load pickled files
data = {}
for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [ ]:
# Double check to make sure data has been loaded properly
data.keys()

In [ ]:
# More checks
data['louis'][:2]

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [ ]:
# Let's take a look at our data again
next(iter(data.keys()))

In [ ]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
next(iter(data.values()))

In [ ]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [ ]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [ ]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

In [ ]:
# Let's take a look at the transcript for Ali Wong
data_df.transcript.loc['ali']

In [ ]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [ ]:
# Let's take a look at our dataframe
data_df

In [ ]:
# Let's add the comedians' full names as well
full_names = ['Ali Wong', 'Anthony Jeselnik', 'Bill Burr', 'Bo Burnham', 'Dave Chappelle', 'Hasan Minhaj',
              'Jim Jefferies', 'Joe Rogan', 'John Mulaney', 'Louis C.K.', 'Mike Birbiglia', 'Ricky Gervais']

data_df['full_name'] = full_names
data_df

In [ ]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [ ]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

In [ ]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [ ]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

## Additional Exercises

1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?